In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.io
from scipy.signal import resample
from datasetResize import *
from sklearn.metrics import classification_report
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder


In [13]:
# lecture du excel
labels_df = pd.read_csv('REFERENCE-V3.csv', header=None)
labels_df.columns = ['filename', 'label']

print(labels_df.head())

  filename label
0   A00001     N
1   A00002     N
2   A00003     N
3   A00004     A
4   A00005     A


In [14]:
# Charger tous les signaux et leurs labels
labels = []
signals = []
signals_padded = []
signals_truncated = []

# Initialiser avec les paramètres pour le redimensionnement
ecg_resizer_max = ECGResizing(target_length=18286)
ecg_resizer_med = ECGResizing(target_length=9000)

for index, row in labels_df.iterrows():
    
    labels.append(row['label'])
    filename = row['filename']
    # Charger un signal ECG
    signal = ecg_resizer_max.load_ecg(f'training2017/{filename}.mat')
    #signals = np.append(signals,[[signal]])
    signals.append(signal)

    # Appliquer le zero padding
    signal_padded = ecg_resizer_max.resize_signal(signal, method='padding')
    signals_padded.append(signal_padded)

    # Appliquer le tronquage
    signal_truncated = ecg_resizer_med.resize_signal(signal, method='padding')
    signals_truncated.append(signal_truncated)



In [15]:
class ECG_RNN:
    def __init__(self, input_shape, num_classes):
        """
        Initialize the ECG_RNN model.
        
        Parameters:
        input_shape (tuple): Shape of the input data (timesteps, features).
        num_classes (int): Number of output classes.
        """
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self.build_model()

    def build_model(self):
        """
        Build the RNN model.
        
        Returns:
        model: Compiled RNN model.
        """
        model = Sequential()
        model.add(SimpleRNN(64, activation='relu', input_shape=self.input_shape))
        model.add(Dropout(0.2))  # Regularization to prevent overfitting
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.num_classes, activation='softmax'))  # Multi-class classification
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model


    def train(self, X, y, epochs=50, batch_size=32):
        """
        Train the RNN model.
        
        Parameters:
        X (np.array): Input data.
        y (np.array): Labels.
        epochs (int): Number of training epochs.
        batch_size (int): Size of training batches.
        """
        self.model.fit(X, y, epochs=epochs, batch_size=batch_size)

    def evaluate(self, X, y):
        """
        Evaluate the model on test data.
        
        Parameters:
        X (np.array): Input test data.
        y (np.array): Test labels.
        
        Returns:
        loss, accuracy: Loss and accuracy of the model on test data.
        """
        return self.model.evaluate(X, y)

    def predict(self, X):
        """
        Predict classes for new data.
        
        Parameters:
        X (np.array): Input data for prediction.
        
        Returns:
        np.array: Predicted class probabilities.
        """
        return self.model.predict(X)


In [16]:
# Normaliser et préparer les signaux
X = np.array(signals_truncated)  # Convertir la liste en tableau NumPy
X = X.reshape(X.shape[0], X.shape[1], 1)  # Reshape pour RNN (samples, timesteps, features)

# Encoder les étiquettes
le = LabelEncoder()
y_encoded = le.fit_transform(labels)  # Encodage initial des étiquettes
y = tf.keras.utils.to_categorical(y_encoded)  # Conversion en one-hot encoding

# Séparer les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
# Définir la forme d'entrée et le nombre de classes
input_shape = (X.shape[1], 1)  # (timesteps, features)
num_classes = y.shape[1]  # Nombre de classes

# Créer une instance de ECG_RNN
ecg_rnn = ECG_RNN(input_shape=input_shape, num_classes=num_classes)

# Entraîner le modèle
ecg_rnn.train(X_train, y_train, epochs=5, batch_size=32)


Epoch 1/5
214/214 ━━━━━━━━━━━━━━━━━━━━ 123s 573ms/step - accuracy: 0.4606 - loss: 3.4123
Epoch 2/5
214/214 ━━━━━━━━━━━━━━━━━━━━ 93s 435ms/step - accuracy: 0.5295 - loss: 1.6454
Epoch 3/5
214/214 ━━━━━━━━━━━━━━━━━━━━ 101s 471ms/step - accuracy: 0.5495 - loss: 1.4070
Epoch 4/5
214/214 ━━━━━━━━━━━━━━━━━━━━ 105s 489ms/step - accuracy: 0.5541 - loss: 1.3254
Epoch 5/5
214/214 ━━━━━━━━━━━━━━━━━━━━ 108s 504ms/step - accuracy: 0.5701 - loss: 1.1720


In [19]:
import numpy as np
from sklearn.metrics import classification_report

# Supposons que X_test est tes données de test
# Effectuer des prédictions sur les données de test
predicted_probs = ecg_rnn.model.predict(X_test)

# Convertir les probabilités en classes prédites
predicted_classes = np.argmax(predicted_probs, axis=1)

# Si y_test est en one-hot encoding, convertir aussi en classes
y_test_encoded = np.argmax(y_test, axis=1)

# Générer le rapport de classification
report = classification_report(y_test_encoded, predicted_classes, target_names=['N', 'A', 'O', '~'])
print(report)


54/54 ━━━━━━━━━━━━━━━━━━━━ 8s 143ms/step
              precision    recall  f1-score   support

           N       0.00      0.00      0.00       140
           A       0.61      0.99      0.76      1044
           O       0.39      0.02      0.04       473
           ~       0.00      0.00      0.00        49

    accuracy                           0.61      1706
   macro avg       0.25      0.25      0.20      1706
weighted avg       0.48      0.61      0.47      1706



c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Albus\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
